<a href="https://colab.research.google.com/github/SatoJin02/BDA_course25/blob/main/Ex03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!cp "/content/drive/MyDrive/data.zip" "/content"

#Step 1: Download and Upload Data

We obtained the monthly air quality monitoring data from the official Japanese environmental database:
Soramame

The downloaded file is a ZIP archive containing multiple CSV files, each representing data from different prefectures or stations.
This ZIP file is uploaded to Google Colab for analysis.

In [2]:
# =======================================
# 1. 必要ライブラリのインストールとインポート
# =======================================
!pip install pandas pandasql

import pandas as pd
import zipfile
import os
from pandasql import sqldf


pysqldf = lambda q: sqldf(q, globals())

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=60ccc9ae21faa08e85c4cfecc8578a2b121f9c5a080b34056f7d5037eaf0ac98
  Stored in directory: /root/.cache/pip/wheels/15/a1/e7/6f92f295b5272ae5c02365e6b8fa19cb93f16a537090a1cf27
Successfully built pandasql


#Step 2: Unzip the File

We use Python’s built-in zipfile module to extract all the CSV files from the ZIP archive into a local directory (/content/data).

In [3]:
# =======================================
# 2. ZIPファイルの展開
# =======================================
zip_path = "/content/data.zip"
extract_dir = "/content/data"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Extracted files:", len(os.listdir(extract_dir)))

Extracted files: 1686


#Step 3: Load All CSV Files into a Single DataFrame

Each CSV file contains similar column structures, such as station code (測定局コード), pollutant names (e.g., PM2.5), and timestamps.
We read all CSVs using pandas.read_csv() and then concatenate them into one unified DataFrame using pd.concat().

In [4]:
# =======================================
# 3. すべてのCSVを1つのDataFrameに結合
# =======================================
all_files = [os.path.join(extract_dir, f) for f in os.listdir(extract_dir) if f.endswith(".csv")]
dfs = []

for file in all_files:
    try:
        df = pd.read_csv(file, encoding="shift-jis", low_memory=False)
        dfs.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

if dfs:
    full_df = pd.concat(dfs, ignore_index=True)
    print("DataFrame created successfully.")
else:
    print("No CSV files found.")
    full_df = pd.DataFrame()

print("DataFrame shape:", full_df.shape)
full_df.head()

DataFrame created successfully.
DataFrame shape: (1168614, 19)


,測定局コード,日付,時,SO2(ppm),NO(ppm),NO2(ppm),NOx(ppm),CO(ppm),Ox(ppm),NMHC(ppmC),CH4(ppmC),THC(ppmC),SPM(mg/m3),PM2.5(μg/m3),SP(mg/m3),WD(16Dir),WS(m/s),TEMP(℃),HUM(％)
0,28204020,2025/09/01,1,NaN,NaN,NaN,NaN,NaN,0.017,NaN,NaN,NaN,NaN,NaN,NaN,WSW,2.0,29.6,75.0
1,28204020,2025/09/01,2,NaN,NaN,NaN,NaN,NaN,0.017,NaN,NaN,NaN,NaN,NaN,NaN,SW,2.7,29.5,75.0
2,28204020,2025/09/01,3,NaN,NaN,NaN,NaN,NaN,0.014,NaN,NaN,NaN,NaN,NaN,NaN,SW,3.4,29.2,75.0
3,28204020,2025/09/01,4,NaN,NaN,NaN,NaN,NaN,0.014,NaN,NaN,NaN,NaN,NaN,NaN,SW,3.3,29.0,77.0
4,28204020,2025/09/01,5,NaN,NaN,NaN,NaN,NaN,0.012,NaN,NaN,NaN,NaN,NaN,NaN,SW,3.5,29.0,77.0


In [5]:
# =======================================
# 4. SQLを使ったデータ分析 (pandasql)
# =======================================
# ---------------------------------------
# 4-1. 個別センサーの総数を求める
# ---------------------------------------

sensor_cols = [col for col in full_df.columns if "測定局" in col or "局コード" in col]
if sensor_cols:
    sensor_col = sensor_cols[0]
    query_sensors = f"""
    SELECT COUNT(DISTINCT [{sensor_col}]) AS total_sensors
    FROM full_df
    """
    sensors_df = pysqldf(query_sensors)
    print("Total individual sensors:")
    display(sensors_df)
else:
    print("No sensor code column found. Please check the CSV column names.")
    print("Available columns:", list(full_df.columns)[:15])

Total individual sensors:


,total_sensors
0,1645


#Step 5: Perform SQL Queries with pandasql

Using the pandasql library, we can write SQL-like queries directly on the pandas DataFrame.
We perform two key analyses:

Count the total number of unique sensors (測定局コード).
This helps identify how many individual measurement stations reported data during the month.

Find the minimum, maximum, and average values of PM2.5.
This shows the air quality range across all recorded sensors.

In [6]:
# ---------------------------------------
# 4-2. PM2.5 の最小値・最大値・平均値
# ---------------------------------------
pm25_cols = [col for col in full_df.columns if "PM2.5" in col or "ＰＭ２．５" in col]
if pm25_cols:
    pm25_col = pm25_cols[0]
    query_pm25 = f"""
    SELECT
        MIN([{pm25_col}]) AS min_pm25,
        MAX([{pm25_col}]) AS max_pm25,
        AVG([{pm25_col}]) AS avg_pm25
    FROM full_df
    WHERE [{pm25_col}] IS NOT NULL
    """
    pm25_df = pysqldf(query_pm25)
    print(f"PM2.5 statistics for column '{pm25_col}':")
    display(pm25_df)
else:
    print("PM2.5 column not found in dataset. Please check column names.")
    print("Available columns:", list(full_df.columns)[:15])

PM2.5 statistics for column 'PM2.5(μg/m3)':


,min_pm25,max_pm25,avg_pm25
0,-,99,7.022433
